# Introducing Meta Llama 3.1

[Workers AI](https://developers.cloudflare.com/workers-ai) allows you to run machine learning models, on the Cloudflare network, from your own code – whether that be from Workers, Pages, or anywhere via REST API.

This notebook will explore the Workers AI REST API using the [official Python SDK](https://github.com/cloudflare/cloudflare-python).

📓 [Open in Google Colab](https://colab.research.google.com/github/craigsdennis/notebooks-cloudflare-workers-ai/blob/main/llama-3.1.ipynb)

In [1]:
import sys
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install -U cloudflare 


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
import os
from getpass import getpass

from cloudflare import Cloudflare
from IPython.display import display, Image, Markdown, Audio
import requests

In [3]:
%load_ext dotenv
%dotenv

## Configuring your environment

To use the API you'll need your [Cloudflare Account ID](https://dash.cloudflare.com). Head to AI > Workers AI page and press the "Use REST API". This page will let you create a new API Token and copy your Account ID.

If you want to add these values to your environment variables, you can **create a new file** named `.env` and this notebook will read those values.

```bash
CLOUDFLARE_API_TOKEN="YOUR-TOKEN"
CLOUDFLARE_ACCOUNT_ID="YOUR-ACCOUNT-ID"
```

Otherwise you can just enter the values securely when prompted below.

In [4]:
if "CLOUDFLARE_API_TOKEN" in os.environ:
    api_token = os.environ["CLOUDFLARE_API_TOKEN"]
else:
    api_token = getpass("Enter you Cloudflare API Token")

In [5]:
if "CLOUDFLARE_ACCOUNT_ID" in os.environ:
    account_id = os.environ["CLOUDFLARE_ACCOUNT_ID"]
else:
    account_id = getpass("Enter your account id")

In [6]:
# Initialize client
client = Cloudflare(api_token=api_token)

## Advanced logic and reasoning

In [7]:
model_name = "@cf/meta/llama-3.1-8b-instruct"

In [8]:
result = client.workers.ai.run(
    model_name,
    account_id=account_id,
    messages = [
        {"role": "system", "content": """You think through puzzles step by step and share your train of thought. 
            The user will present a puzzle and you will explain your thinking, and then provide your best guess."""},
        {"role": "user", "content": """I have three shells in a line. I hide a dollar under the shell in the middle. 
        
        When I swap the shells, I bring the dollar with the shell. 
        
        After each swap, the shells will have a new position, and I will refer to them with their current placement in a line.
        
        I swap the middle shell with the one on the left, leaving the right shell alone.

        Then I swap shells in the middle and the right, leaving the left shell alone.

        Finally I swap right and left and leave the middle alone.

        Where is the dollar?"""}
    ]
)

# Shhhhh! It should be under the right shell
display(Markdown(result["response"]))

Let's break down each step and track the movement of the dollar. Initial position:Shell 1 on the left, Shell 2 in the middle, Shell 3 on the right, and the dollar under Shell 2.Step 1: Swap Shell 1 and Shell 2 (leave Shell 3 alone)New positions: Shell 2 on the left, Shell 1 in the middle, Shell 3 on the right. The dollar stays with Shell 1 (or was previously above it, now above Shell 2). Step 2: Swap Shell 2 and Shell 3 (leave Shell 1 alone)New positions: Shell 1 on the left, Shell 3 in the middle, Dollar-swap Shell 2 on the right. Dollar remains in the middle shell now. Step 3: Swap Shell 1 and Shell 3 (leave Shell 2 alone)New positions: Original Dollar-middle Shell on the left, and previous second shell on the right. The dollar must now be left in an original middle shell position.

### Tool use

Function calling is a powerful way to allow your LLM to call functions and process their responses.

In [9]:
hello_count = 0 

def hello_world(name="World"):
    global hello_count
    hello_count = hello_count + 1
    return json.dumps({"message": f"Hello, {name}!", "greetCount": hello_count})

In [11]:
messages = [
    {"role": "system", "content": """
        You are a greeter.
        
        The more you greet a person, the more close you become. Acknowledge the number of times you have met before.

        Describe your feelings towards the person in a detailed way through similie and metaphor. 
        """},
    {"role": "user", "content": "Say hello to Bob"}
]
tools = [
    {
        "function": {   
            "name": "helloWorld",
            "description": "Greets a person",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": { "type": 'string', "description": "the name to be greeted"},
                },
            },
        },
        "type": "function"
    },
]

result = client.workers.ai.run(
    model_name,
    account_id=account_id,
    messages=messages,
    tools=tools
)

while "tool_calls" in result:
    for tool_call in result["tool_calls"]:
        fn_response = None
        print(f"tool_call: {tool_call}")
        if tool_call["name"] == "helloWorld":
            fn_response = hello_world(**tool_call["arguments"])
        if fn_response is not None:
            print(f"fn_response: {fn_response}")
            messages.append({"role": "tool", "content": fn_response})
            # print(f"messages: {messages}")
            result = client.workers.ai.run(
                model_name,
                account_id=account_id,
                messages=messages,
                tools=tools
            )
print(result["response"])

tool_call: {'name': 'helloWorld', 'arguments': {'name': 'Bob'}}
fn_response: {"message": "Hello, Bob!", "greetCount": 8}
tool_call: {'name': 'helloWorld', 'arguments': {'name': 'Bob'}}
fn_response: {"message": "Hello, Bob!", "greetCount": 9}
tool_call: {'name': 'helloWorld', 'arguments': {'name': 'Bob'}}
fn_response: {"message": "Hello, Bob!", "greetCount": 10}


KeyboardInterrupt: 

## Additional language support

In [ ]:
supported_languages = ["English", "German", "French", "Italian", "Portuguese", "Hindi", "Spanish", "Thai"]
prompts = []
for lang in supported_languages:
    result = client.workers.ai.run(
        model_name,
        account_id=account_id,
        prompt=f"""
        Generate instructions in {lang} that could be used to instruct an LLM to create an alliterative sentence in {lang}.
        
        Do not include any preamble or introduction, return only the prompt."""
    )
    prompt = result["response"]
    print(f"{lang}: {prompt}\n\n")
    prompts.append(prompt)

In [ ]:
for prompt in prompts:
    result = client.workers.ai.run(
        model_name,
        account_id=account_id,
        prompt=prompt,
    )
    print(result["response"])
    print("\n\n")